# WGAN-GP生成二次元原理与解析

## 查看代码时可以查看一下这篇论文[论文地址](https://arxiv.org/pdf/1704.00028.pdf)，论文讲的就是wgan-gp，如果你没有gan的基础可以先了解了解gan的原理，实际上gan发展到wgan-gp在代码层面其实改变的并不多，后面我会对很多gan进行开源，望大家支持，谢谢！

## WGAN-GP较前面的模型有哪些改进呢？
1. 这次的模型，我们依然使用了`DCGAN`的网络结构，因为`WGAN-GP`的学习重点不在网络上

2. WGAN 论文发现了 JS 散度导致 GAN 训练不稳定的问题，并引入了一种新的分布距离度量方法：Wasserstein 距离，也叫推土机距离(Earth-Mover Distance，简称 EM 距离)，它表示了从一个分布变换到另一个分布的最小代价

3. 由于前WGAN并没有真正的实现`1-Lipschitz`，只有对任意输入x梯度都小于或等于1的时候，则该函数才是` 1-Lipschitz function`

##  最后得到本篇精髓
![](https://ai-studio-static-online.cdn.bcebos.com/1ffe098b66e7463f9c938b343f3d3f5685213396c9c448d8bee984746b51dcef)
### **也就是想尽办法的让判别器认为真实的图片得到更高的分数所以需要加上负号，让生成的图片得到更低的分数，最后加上一个梯度的惩罚项，对于惩罚项的解释(在惩罚项中希望,如果越是满足`1-Lipschitz function`，惩罚就越少。事实证明这样做的效果是非常好的),现在大量GAN都能看到WGAN-GP的身影，所以它非常的重要**
![](https://ai-studio-static-online.cdn.bcebos.com/ca021e41c11242eaafd52f8a5b2b13ca90a88b2a4f044bb59bc0720e84d85d25)
### **其中有个超参数（1）表示超参数  本文设置为10**

In [1]:
# 导入包paddle几乎都这么导
import paddle.fluid as fluid
import paddle
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random

###  生成器的实现，网络可以自行实现我这个为基本版随意写的，主要是领略GAN的思想，也是为了方便学习
1. paddle中的静态图如果需要求导必须给定参数的名称，所以必须给每个参数命名一个独一无二的名字此处使用到了`fluid.unique_name.guard`用于生成参数名
2. 生成器使用到了逆转卷积`conv2d_transpose`顾名思义是一种逆转的卷积和卷积正好相反，使用到了卷积这就是`DCGAN`与`传统GAN`的区别
3. `conv2d_transpose`的paddling为`SAME`的时候有个非常简单的计算公式，图的大小 * 步长，为了方便我直接用的步长为2

## 网络结构
* 1. 使用卷积网络进行下采样
* ![](https://ai-studio-static-online.cdn.bcebos.com/0cb758271fde40b68d51642452f8b18f6b343dd78b8c4fc09c031c0d8db85a09)
* 2. 使用反卷积进行上采样
* ![](https://ai-studio-static-online.cdn.bcebos.com/addd8ef2fd7b402e9d56ea35d98b0666fa77f74feca4438f9af3d38421d01081)
* 取消所有 pooling 层。G 网络中使用微步幅度卷积（fractionally strided convolution）
* 代替 pooling 层，D 网络中使用步幅卷积（strided convolution）代替 pooling 层。 ·在 D 和 G 中均使用 batch normalization
* 去掉 FC 层，使网络变为全卷积网络
* G 网络中使用 ReLU 作为激活函数，最后一层使用 tanh
* D 网络中使用 LeakyReLU 作为激活函数

# 于是按照给定的网络结构可以写出适合自己图片大小的网络

In [2]:
# 3*96*96
def Generator(x, name='G'):
    # (-1,100,1,1)
    def deconv(x, num_filters, filter_size=5,padding='SAME', stride=2, act='relu'):
            x = fluid.layers.conv2d_transpose(x, num_filters=num_filters, filter_size=filter_size, stride=stride, padding=padding)
            x = fluid.layers.batch_norm(x, momentum=0.8)
            if act=='relu':
                x = fluid.layers.relu(x)
            elif act=='tanh':
                x = fluid.layers.tanh(x)
            return x
    with fluid.unique_name.guard(name+'/'):
        x = fluid.layers.reshape(x, (-1, 100, 1, 1))
        x = deconv(x, num_filters=512, filter_size=6, stride=1, padding='VALID') # 6
        x = deconv(x, num_filters=256) # 12
        x = deconv(x, num_filters=128) #24
        x = deconv(x, num_filters=64) #48
        x = deconv(x, num_filters=3, act='tanh') #96
    return x

### 判别器与生成器差不多
1. 就是将生成器反过来进行运算，最后使用全连接输出1维度的数即可

In [3]:
def Discriminator(x, name='D'):
    def conv(x, num_filters, momentum=0.8):
            x = fluid.layers.conv2d(x, num_filters=num_filters, filter_size=5, stride=2, padding='SAME')
            x = fluid.layers.batch_norm(x, momentum=momentum)
            x = fluid.layers.leaky_relu(x, alpha=0.2)
            x = fluid.layers.dropout(x, dropout_prob=0.25)
            return x
    with fluid.unique_name.guard(name+'/'):
        # (3, 96, 96)
        x = conv(x, num_filters=64) # 48
        x = conv(x, num_filters=128) # 24
        x = conv(x, num_filters=256) # 12
        x = conv(x, num_filters=512) # 6
        x = fluid.layers.pool2d(x, pool_type='avg', global_pooling=True)
        x = fluid.layers.flatten(x)
        x = fluid.layers.fc(x, size=1)
    return x

### 静态图必须进行的步揍不多解释

In [4]:
d_program_1 = fluid.Program()
d_program_2 = fluid.Program()
g_program = fluid.Program()
# 每批次数量
batch_size = 128

### 判别器优化器实现
1. 生成器的超参数有噪声的数量这里固定为100
2. 通过输入的噪声生成图像(`Generator`)，并将生成的图片给判别器(`Discriminator`),我们给它一个低分
3. 直接将真实的图片给判别器，并给它一个高分，由于优化器是梯度下降的方向，我们想要得到上升那么就给损失一个负号用于得到高分
4. 给定真实图片和生成图片，完成梯度惩罚项GP, 并让他减小损失
5. 训练需要固定生成器，让判别器学习
6. 优化器 beta1=0, beta2=0.9
7. 关于GP梯度惩罚项的实现[参考](https://github.com/PaddlePaddle/models/blob/release/1.7/PaddleCV/gan/trainer/STGAN.py第150行的函数)
8. c为梯度惩罚项的超参数默认为 10.0

In [5]:
z_num = 100
c = 10.0
with fluid.program_guard(d_program_1):
    # 传入参数用fluid.data
    fake_z_1 = fluid.data(name='fake_z', dtype='float32', shape=(None, z_num))
    # 通过生成器生成图片
    img_fake_1 = Generator(fake_z_1)
    # 判别器判断好坏
    fake_ret_1 = Discriminator(img_fake_1)
    # 判别器损失
    loss_1 = fluid.layers.mean(fake_ret_1)

    # 将名字为开头为D的参数取出来进行优化，也就是判别器的简写
    parameter_list = []
    for var in d_program_1.list_vars():
        if fluid.io.is_parameter(var) and var.name.startswith('D'):
            parameter_list.append(var.name)
    
    # 优化一下，优化原理为将判别器错误的图片尽可能的判别为小的
    fluid.optimizer.AdamOptimizer(learning_rate=1e-4,beta1=0, beta2=0.9).minimize(loss_1, parameter_list=parameter_list)

    # 用fluid.data传入真实的图片
    img_real_1 = fluid.data(name='img_real', dtype='float32', shape=(None, 3, 96, 96))
    # 用判别真实的图片
    real_ret_1 = Discriminator(img_real_1)
    # 损失函数
    loss_2 = -1 * fluid.layers.mean(real_ret_1)
    # 为啥要加个负号呢？因为我们想让真实的图片分数尽可能的大，优化器是让损失减小，加个负号即是增大
    fluid.optimizer.AdamOptimizer(learning_rate=1e-4,beta1=0, beta2=0.9).minimize(loss_2, parameter_list=parameter_list)
    
    # 这里的惩罚项gp是copy于paddle的github
    # https://github.com/PaddlePaddle/models/blob/release/1.7/PaddleCV/gan/trainer/STGAN.py第150行的函数
    def _interpolate(a, b=None):
        beta = fluid.layers.uniform_random_batch_size_like(
            input=a, shape=a.shape, min=0.0, max=1.0)
                
        mean = fluid.layers.reduce_mean(
            a, dim=list(range(len(a.shape))), keep_dim=True)
        input_sub_mean = fluid.layers.elementwise_sub(a, mean, axis=0)
        var = fluid.layers.reduce_mean(
            fluid.layers.square(input_sub_mean),
            dim=list(range(len(a.shape))),
            keep_dim=True)
        b = beta * fluid.layers.sqrt(var) * 0.5 + a
        shape = [a.shape[0]] 
        alpha = fluid.layers.uniform_random_batch_size_like(
            input=a, shape=shape, min=0.0, max=1.0)

        inner = fluid.layers.elementwise_mul((b-a), alpha, axis=0) + a
        return inner

    x = _interpolate(img_real_1, img_fake_1)
    pred = Discriminator(x)
    
    vars = []
    for var in fluid.default_main_program().list_vars():
        if fluid.io.is_parameter(var) and var.name.startswith("D"):
            vars.append(var.name)
    grad = fluid.gradients(pred, x, no_grad_set=vars)[0]
    grad_shape = grad.shape
    grad = fluid.layers.reshape(
        grad, [-1, grad_shape[1] * grad_shape[2] * grad_shape[3]])
    epsilon = 1e-16
    norm = fluid.layers.sqrt(
        fluid.layers.reduce_sum(
            fluid.layers.square(grad), dim=1) + epsilon)
    gp = fluid.layers.reduce_mean(fluid.layers.square(norm - 1.0)) * c
    d_loss = loss_1 + loss_2 + gp
    # clip = fluid.clip.GradientClipByValue(min=CLIP[0], max=CLIP[1])
    # 优化即可
    fluid.optimizer.AdamOptimizer(learning_rate=1e-4,beta1=0, beta2=0.9).minimize(gp, parameter_list=parameter_list)

### 生成器优化的实现
1. 输入噪声给生成器，因为我们想让生成器生成一张真实的图片，所以我们需要固定判别器，让生成器生成一张趋于真实的图片，所以要让他向分数高的方向走，所以也要加个负号

In [6]:
with fluid.program_guard(g_program):
    fake_z_2 = fluid.data(name='fake_z', dtype='float32', shape=(None, z_num))
    
    img_fake_2 = Generator(fake_z_2)
    # 克隆一下，并固定参数，用于预测
    test_program = g_program.clone(for_test=True)
    fake_ret_2 = Discriminator(img_fake_2)
    

    # loss = fluid.layers.sigmoid_cross_entropy_with_logits(x=fake_ret, label=fluid.layers.ones_like(real_ret))
    avg_loss_2 = -1 * fluid.layers.mean(fake_ret_2)
    
    parameter_list = []
    for var in d_program_1.list_vars():
        if fluid.io.is_parameter(var) and var.name.startswith('G'):
            parameter_list.append(var.name)
    
    # 这里是让分数尽可能的大，所以加上负号
    fluid.optimizer.AdamOptimizer(learning_rate=1e-4, beta1=0, beta2=0.9).minimize(avg_loss_2, parameter_list=parameter_list)

### 准备训练前的配置

In [ ]:
use_gpu = True
if use_gpu:
    place = fluid.CUDAPlace(0)
else:
    place = fluid.CPUPlace()

exe = fluid.executor.Executor(place=place)
exe.run(fluid.default_startup_program())

### 数据读入函数

In [6]:
import os
!unzip -oq /home/aistudio/data/data17962/二次元人物头像.zip
fullpath = '/home/aistudio/faces'
def get_batch():
    filenames = os.listdir(fullpath)
    random.shuffle(filenames)
    img_list = []
    label_list = []
    for i, filename in enumerate(filenames):
        fullname = os.path.join(fullpath, filename)
        img = cv2.imread(fullname, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img / 255.*2 - 1
        img = np.transpose(img, (2,0,1))
        img_list.append(img)
        label_list.append(1.)
        if (i+1) % batch_size == 0:
            yield np.array(img_list).astype('float32'), np.array(label_list)
            img_list = []
            label_list = []

### 快照继续训练

In [2]:
try:
    fluid.io.load_params(exe, dirname='/home/aistudio/out_params', main_program=fluid.default_startup_program())
except:
    pass
def get_cur():
    try:
        img_names = np.array([i.strip('.jpg').split('_') for i in os.listdir('/home/aistudio/out_params/') if '.jpg' in i]).astype('int')
    except:
        return [0, 0]
    if len(img_names) == 0:
        return [0, 0]
    return np.max(img_names, axis=0)
cur_process = get_cur()
epoch_pro = cur_process[0]
step_pro = cur_process[1]

### 训练开始
1. 由于判别器学习得快一点所以需要，为了保持同步我们可以将生成器进行多次得训练

In [3]:
epochs = 20000
train_d = 2
for epoch in range(epochs):
    for step,(x, y) in enumerate(get_batch()):
        epoch += epoch_pro
        step += step_pro
        fake_z = np.random.uniform(size=(x.shape[0], z_num), low=-1, high=1).astype('float32')
        [loss_1_, loss_2_, gp_] = exe.run(program=d_program_1, 
                                feed={'fake_z':fake_z,'img_real':x}, 
                                fetch_list=[loss_1, loss_2, gp])


        # 生成器训练
        for _ in range(train_d):
            fake_z = np.random.uniform(size=(x.shape[0], z_num), low=-1, high=1).astype('float32')
            [g_loss] = exe.run(program=g_program, 
                    feed={'fake_z':fake_z}, 
                    fetch_list=[avg_loss_2])
        
        # 100次进行预测一次
        if step % 100 == 0:
            print(loss_1_, loss_2_, gp_, g_loss)
            print('[Training] epoch:{} step:{} d_loss:{} g_loss:{}'.format(epoch, step, loss_1_+loss_2_+gp_, g_loss))
            # 快照
            fluid.io.save_params(exe, dirname='/home/aistudio/out_params', main_program=fluid.default_startup_program())    
            fake_z = np.random.uniform(size=(z_num, z_num), low=-1, high=1).astype('float32')
            [pre_im] = exe.run(program=test_program, 
                    feed={'fake_z':fake_z}, 
                    fetch_list=[img_fake_2])
            pre_im = (np.transpose(pre_im, (0, 2, 3, 1))+1) / 2
            
            # 准备一个画布用于存放生成的图片
            images = np.zeros((960, 960, 3))
            for h in range(10):
                for w in range(10):
                    images[h*96:(h+1)*96, w*96:(w+1)*96] = pre_im[(h*10)+w]
            plt.imsave('/home/aistudio/out_params/{}_{}.jpg'.format(epoch, step), images)
            plt.imshow(images, cmap='gray')
            plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/home/aistudio/faces'

In [ ]:
import os
import cv2

### 最后可以将我们得图片生成过程做成视频，感受GAN带来得神奇

In [ ]:
video_dir = './a.mp4'
fps = 5
img_size = (960, 960)

fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
video_writer = cv2.VideoWriter(video_dir, fourcc, fps, img_size)
file_list = os.listdir('out_params')
file_list_ = []
for i in file_list:
    if '.jpg' in i:
        epoch, step = i.strip('.jpg').split('_')
        file_list_.append(int(epoch) * 1000 + int(step))
file_list_.sort()
for i in file_list_:
    img_name = os.path.join('out_params', '%s_%s' % (i // 1000, i % 1000)+".jpg")
    img_array = cv2.imread(img_name, cv2.IMREAD_COLOR)
    video_writer.write(img_array)
video_writer.release()

# 简单总结一下吧
1. 也是第一次在paddle上写完整的网络，通过查文档问老师提issue，也是在大家的帮助下终于完成了项目
2. 就本次实验来说，让我更加了解WGAN-GP的实现,也为是实现其他GAN模型奠定了基础
3. 但是实验过程中我发现，生成效果并不是非常好，我猜想这和网络有直接关系，希望可以阅读跟多的文章借鉴跟多文章来生成更高清的二次元
# 介绍自己
* 喜欢编程，能全栈
* 选择Python，更热爱AI
*  来AI Studio互粉吧~等你哦~ [主页](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/340127)